In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark Example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
spark

In [ ]:
accounts = spark.read.option('header',True).csv('./data/accounts.csv')
accounts

In [ ]:
accounts.printSchema()

In [ ]:
transactions = spark.read.option('header', True).parquet('./data/transactions.parquet')

In [ ]:
transactions.count()

In [ ]:
account_transactions = transactions.groupby('account_number').sum()
with_sum = accounts.join(account_transactions, 'account_number', 'inner')
accounts = with_sum.withColumn('new_balance', sum([with_sum.balance, with_sum['sum(amount)']]))

In [ ]:
accounts.printSchema()

In [ ]:
neg_balance = accounts.filter(accounts.new_balance < 0)

In [ ]:
clients = spark.read.json('./data/clients.json')

In [ ]:
clients.printSchema()

In [ ]:
clients = clients.join(neg_balance, 'account_number', 'inner')

In [ ]:
clients.select(['first_name','last_name', 'account_number','new_balance']).show(5)